In [1]:
#bibliotecas
import pandas as pd
import pathlib
import smtplib
from email.message import EmailMessage

In [2]:
#base de dados
emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='latin1', sep = ';')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')

In [3]:
#incluir nome da loja em vendas
vendas = vendas.merge(lojas, on='ID Loja')
display(vendas)


,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
3,2,2019-01-02,3,Relógio,3,200,600,Norte Shopping
4,2,2019-01-02,3,Chinelo Liso,1,71,71,Norte Shopping
...,...,...,...,...,...,...,...,...
100994,69996,2019-12-26,17,Short Listrado,2,102,204,Center Shopping Uberlândia
100995,69996,2019-12-26,17,Mochila,4,270,1080,Center Shopping Uberlândia
100996,69996,2019-12-26,17,Pulseira Estampa,1,87,87,Center Shopping Uberlândia
100997,69997,2019-12-26,11,Camisa Listrado,1,108,108,Ribeirão Shopping


In [4]:
#criar um data frame pra cada loja
dicionario_lojas = {}
for loja in lojas['Loja']:
    dicionario_lojas[loja] = vendas.loc[vendas['Loja']== loja, :]#adiciono a no dicionario uma tabela para todas as linhas em que a coluna 'Loja'  for igual a loja achada no for da tabela 'Lojas'(se ja nao existir algum item com essa loja)
#display(dicionario_lojas['nome da loja desejada])

for item in dicionario_lojas:
    print(item)

    

Iguatemi Esplanada
Shopping Midway Mall
Norte Shopping
Shopping Iguatemi Fortaleza
Shopping União de Osasco
Shopping Center Interlagos
Rio Mar Recife
Salvador Shopping
Rio Mar Shopping Fortaleza
Shopping Center Leste Aricanduva
Ribeirão Shopping
Shopping Morumbi
Parque Dom Pedro Shopping
Bourbon Shopping SP
Palladium Shopping Curitiba
Passei das Águas Shopping
Center Shopping Uberlândia
Shopping Recife
Shopping Vila Velha
Shopping SP Market
Shopping Eldorado
Shopping Ibirapuera
Novo Shopping Ribeirão Preto
Iguatemi Campinas
Shopping Barra


In [5]:
#pegando dia do indicador
dia_indicador = vendas['Data'].max()
print(dia_indicador)

2019-12-26 00:00:00


In [6]:
#salvar planilha na pasta de backup
#verificar se ja existe um arquivo com o mesmo nome

caminho_backup = pathlib.Path(r'Backup Arquivo Lojas')#caminho da pasta

arquivos_backup = caminho_backup.iterdir()#ve os arquivos e pastas dentro da pasta que o caminho leva

lista_nomes_backup = []

lista_nomes_backup = [arquivo.name for arquivo in arquivos_backup]#pra cada arquivo no arquivos_backup ele pega o nome e adiciona na lista_nomes_backup

#criar os pasta pra cada loja

for loja in dicionario_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()

#criando os arquivos pra colocar nas pastas
    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.day, dia_indicador.month, loja)#mostra dia , mes e nome da loja
    local_arquivo = caminho_backup / loja / nome_arquivo
    dicionario_lojas[loja].to_excel(local_arquivo)#criou o arquivo da tabela da maior data encontrada
    

In [7]:
#metas
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

In [ ]:
for loja in dicionario_lojas:
    #calcular indicadores
    #faturamento
    vendas_loja = dicionario_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :]

    #faturamento ano
    faturamento_ano = vendas_loja['Valor Final'].sum()
    #faturamento dia
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()

    #diversidade de produtos

    #diversidade ano
    qtde_produtos_ano = len(vendas_loja['Produto'].unique())
    #diversidade dia
    qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())

    #ticket medio
    valor_venda = vendas_loja.groupby('Código Venda').sum(numeric_only=True)
    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum(numeric_only=True)
    #ticket ano
    ticket_medio_ano = valor_venda['Valor Final'].mean()
    #print(ticket_medio_ano)
    #ticket dia
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
    #print(ticket_medio_dia)
    
    
#enviar email
nome = emails.loc[emails['Loja']==loja, 'Gerente'].values[0]  
#.values[0] - para ter apenas o valor #.loc[linha desejada, coluna desejada]
if (faturamento_dia >= meta_faturamento_dia):
    cor_fat_dia = "green"
else:
    cor_fat_dia = "red"
        
if (faturamento_ano >= meta_faturamento_ano):   
    cor_fat_ano = "green"
else:
    cor_fat_ano = "red"
        
if(qtde_produtos_dia >= meta_qtdeprodutos_dia):
    cor_qtde_dia = "green"
else:
    cor_qtde_dia = "red"
        
if(qtde_produtos_ano >= meta_qtdeprodutos_ano):
    cor_qtde_ano = "green"
else:
    cor_qtde_ano = "red"
        
if(ticket_medio_dia >= meta_ticketmedio_dia):
    cor_ticket_dia = "green"    
else:
    cor_ticket_dia = "red"
        
if(ticket_medio_ano >= meta_ticketmedio_ano):
    cor_ticket_ano = "green"
else:
    cor_ticket_ano = "red"
        
msg = EmailMessage()
msg.add_header("Content-type", "text/html")
msg['From'] = 'guigos3456@gmail.com'
msg['To'] = emails.loc[emails['Loja']==loja, 'E-mail'].values[0]
msg['Subject'] = 'OnePage Dia {}/{} - Loja {}'.format(dia_indicador.day, dia_indicador.month, loja)
html = (f"""
    <p>Bom dia, {nome}</p>

    <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>Loja {loja}</strong> foi:</p>

    <table>
    <tr>
        <th>Indicador</th>
        <th>Valor Dia</th>
        <th>Meta Dia</th>
        <th>Cenario Dia</th>
    </tr>
    <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_dia:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
        <td style="text-align: center; color: {cor_fat_dia}">◙</td>
    </tr>
    <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_dia}</td>
        <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
        <td style="text-align: center; color: {cor_qtde_dia}">◙</td>
    </tr>
    <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_dia:.2f}</td>
        <td style="text-align: center; color: {cor_ticket_dia}">◙</td>
    </tr>
    </table>
    <br>
    <table>
    <tr>
        <th>Indicador</th>
        <th>Valor Ano</th>
        <th>Meta Ano</th>
        <th>Cenario Ano</th>
    </tr>
    <tr>
        <td>Faturamento</td>
        <td style="text-align: center">R${faturamento_ano:.2f}</td>
        <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
        <td style="text-align: center; color: {cor_fat_ano}">◙</td>
    </tr>
    <tr>
        <td>Diversidade de Produtos</td>
        <td style="text-align: center">{qtde_produtos_ano}</td>
        <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
        <td style="text-align: center; color: {cor_qtde_ano}">◙</td>
    </tr>
    <tr>
        <td>Ticket Médio</td>
        <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
        <td style="text-align: center">R${meta_ticketmedio_ano:.2f}</td>
        <td style="text-align: center; color: {cor_ticket_ano}">◙</td>
    </tr>
    </table>

    <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>

    <p>Qualquer duvida estou à disposição.</p>

    <p>Att., Eu</p>
                    
                    """)

msg.add_alternative(html, subtype="html")

attachment = pathlib.Path.cwd() / caminho_backup / loja / f'{dia_indicador.month}_{dia_indicador.day}_{loja}.xlsx'

with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
    smtp.login('guigos3456@gmail.com', 'pzbd tnrp mruc dpwp')
    smtp.send_message(msg)
    
print("Email da Loja {} enviado com sucesso.".format(loja))


Email da Loja Iguatemi Esplanada enviado com sucesso.
Email da Loja Shopping Midway Mall enviado com sucesso.
Email da Loja Norte Shopping enviado com sucesso.
Email da Loja Shopping Iguatemi Fortaleza enviado com sucesso.
Email da Loja Shopping União de Osasco enviado com sucesso.
Email da Loja Shopping Center Interlagos enviado com sucesso.
Email da Loja Rio Mar Recife enviado com sucesso.
Email da Loja Salvador Shopping enviado com sucesso.
Email da Loja Rio Mar Shopping Fortaleza enviado com sucesso.
Email da Loja Shopping Center Leste Aricanduva enviado com sucesso.
Email da Loja Ribeirão Shopping enviado com sucesso.
Email da Loja Shopping Morumbi enviado com sucesso.
Email da Loja Parque Dom Pedro Shopping enviado com sucesso.
Email da Loja Bourbon Shopping SP enviado com sucesso.
Email da Loja Palladium Shopping Curitiba enviado com sucesso.
Email da Loja Passei das Águas Shopping enviado com sucesso.
Email da Loja Center Shopping Uberlândia enviado com sucesso.
Email da Loja S

In [24]:
#ranking das lojas no ano
faturamento_lojas_ano = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum(numeric_only=True)
#[['Loja', 'Valor Final']]Lista de colunas para separar apenas o desejado
faturamento_lojas_ano = faturamento_lojas_ano.sort_values(by='Valor Final', ascending=False)
#ordena do menor para o maior mas com ascending=False fica do maior para o menor

nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.day, dia_indicador.month)#mostra dia , mes e nome da loja
faturamento_lojas_ano.to_excel(r'Backup Arquivo Lojas\{}'.format(nome_arquivo))

# ranking das lojas no dia
vendas_dia = vendas.loc[vendas['Data']==dia_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum(numeric_only=True)
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)

nome_arquivo = '{}_{}_Ranking Dia.xlsx'.format(dia_indicador.day, dia_indicador.month)#mostra dia , mes e nome da loja
faturamento_lojas_dia.to_excel(r'Backup Arquivo Lojas\{}'.format(nome_arquivo))

# display(faturamento_lojas_ano)
# display(faturamento_lojas_dia)

#enviar email
body = f'''
Prezados, bom dia

Melhor loja do Dia em faturamento: Loja {faturamento_lojas_dia.index[0]} com faturamento R${faturamento_lojas_dia.iloc[0, 0]:.2f}
Pior loja do Dia em faturamento: Loja {faturamento_lojas_dia.index[-1]} com faturamento R${faturamento_lojas_dia.iloc[-1, 0]:.2f}

Melhor loja do Ano em faturamento: Loja {faturamento_lojas_ano.index[0]} com faturamento R${faturamento_lojas_ano.iloc[0, 0]:.2f}
Pior loja do Ano em faturamento: Loja {faturamento_lojas_ano.index[-1]} com faturamento R${faturamento_lojas_ano.iloc[-1, 0]:.2f}

Segue em anexo o ranking do ano e do dia de todas as lojas.

Qualquer dúvida estou a disposição

Att., 
Eu
'''
        
msg = EmailMessage()
msg.add_header("Content-type", "text/html")
msg['From'] = 'guigos3456@gmail.com'
msg['To'] = emails.loc[emails['Loja']=="Diretoria", 'E-mail'].values[0]
msg['Subject'] = 'Ranking Dia {}/{}'.format(dia_indicador.day, dia_indicador.month)
msg.set_content(body)


attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.day}_{dia_indicador.month}_Ranking Anual.xlsx'
with open(attachment, 'rb') as f:
    msg.add_attachment(
        f.read(),
        maintype='application',
        subtype='octet-stream',
        filename=attachment.name
    )
    
attachment = pathlib.Path.cwd() / caminho_backup / f'{dia_indicador.day}_{dia_indicador.month}_Ranking Dia.xlsx'
with open(attachment, 'rb') as f:
    msg.add_attachment(
        f.read(),
        maintype='application',
        subtype='octet-stream',
        filename=attachment.name
    )

with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
    smtp.login('guigos3456@gmail.com', 'pzbd tnrp mruc dpwp')
    smtp.send_message(msg)
    
print("Email da Diretoria enviado com sucesso.")

Email da Diretoria enviado com sucesso.
